In [50]:
from IPython.display import display
import sys
import pandas as pd
import numpy as np

In [74]:
# Add the Results folders of the different components
# Forward Problem
FP_dir = '/home/sirush/PhDResearch/MMS_Transient_Channel/ForwardProblem/Log_Files/'
InvProb_dir = '/home/sirush/PhDResearch/MMS_Transient_Channel/InverseProblem/Log_Files/'
ODECO_multiple_dir = '/home/sirush/PhDResearch/MMS_Transient_Channel/ODECO/Log_Files/MultipleParameterSpace/'

# Define a list of Re values to keep
Re_to_keep = [60,75,80,100]

# Define the nx/nSteps to keep
nx_to_keep = [72]

# Defien the number of sensors to keep
sensors_to_keep = [3,11]

col_order = ['Source','Re','uErr','Runtime']

In [75]:
# Read in the data from the forward problem
# Get the results cooresponding to the "Generated From" Reynolds numbers
viento_log = 'log_Re{1,20,40,60,80,100}_nxnt{24,36,48,60,72}_short.txt'
viento_data = pd.read_csv(FP_dir+viento_log, sep=' ')

# Get the results cooresponding to the "General" Reynolds numbers
viento_log = 'log_Re{10,25,50,75}_nxnt{24,36,48,60,72}_short.txt'
viento_data = viento_data.append(pd.read_csv(FP_dir+viento_log, sep=' '))

# Filter down the results
viento_data = viento_data[
    viento_data.Re.isin(Re_to_keep) &
    viento_data.nx.isin(nx_to_keep)
    ].sort_values(['Re','nx'])

viento_data = viento_data.drop(['tFinal','nx','nSteps','pErr'],axis=1)
viento_data['Source'] = 'Viento'
viento_data = viento_data[col_order]
viento_data

,Source,Re,uErr,Runtime
19,Viento,60,0.000015,1793.04
19,Viento,75,0.000016,1798.03
24,Viento,80,0.000017,1778.68
29,Viento,100,0.000019,1705.61


In [77]:
# Read in the data from the Inverse Problem
inv_log = 'log_short_Re{1,20,40,60,80,100}_nxnt{24,36,48,60,72}.txt'
inv_data = pd.read_csv(InvProb_dir+inv_log, sep= ' ')
display(inv_data.columns)
# Filter down the results
inv_data = inv_data[
    inv_data.Re.isin(Re_to_keep) &
    inv_data.nx.isin(nx_to_keep)
    ]

inv_data = inv_data.drop(['tFinal','nx','nSteps','alphaErr2','alphaErrInf','uErrInf',
                         'ErrorCheckRunTime','VTKRunTime','Total'],axis=1)
inv_data['Source'] = 'Inverse Problem'
inv_data = inv_data.append(pd.DataFrame({'Source':['Inverse problem'],
                              'Re':75,
                              'InverseTime':np.nan,
                              'uErr2':np.nan}))
inv_data = inv_data.rename(columns={'uErr2':'uErr','InverseTime':'Runtime'})
inv_data = inv_data[col_order].reset_index(drop=True).sort_values('Re')

inv_data

Index([u'tFinal', u'Re', u'nx', u'nSteps', u'alphaErr2', u'alphaErrInf',
       u'uErr2', u'uErrInf', u'InverseTime', u'ErrorCheckRunTime',
       u'VTKRunTime', u'Total'],
      dtype='object')

,Source,Re,uErr,Runtime
0,Inverse Problem,60,0.000258,128.365
3,Inverse problem,75,NaN,NaN
1,Inverse Problem,80,0.000263,128.860
2,Inverse Problem,100,0.000267,128.021


In [78]:
# Read in the data from the ODECO problem solved in the Multiple Parameter Space
ODECO_multiple_log = 'log_short_multiple_ODECO_compare_generated_from.txt'
ODECO_multiple_data = pd.read_csv(ODECO_multiple_dir+ODECO_multiple_log, sep = ' ')


ODECO_multiple_log = 'log_multiple_ODECO_non_generated_short.txt'
ODECO_multiple_data = ODECO_multiple_data.append(pd.read_csv(ODECO_multiple_dir+ODECO_multiple_log, sep=' '))
ODECO_multiple_data = ODECO_multiple_data[
                            ODECO_multiple_data.Re.isin(Re_to_keep) &
                            ODECO_multiple_data.nx.isin(nx_to_keep) &
                            ODECO_multiple_data['numSensors'].isin(sensors_to_keep)
                    ].sort_values(['Re','nx','numSensors'])
ODECO_multiple_data = ODECO_multiple_data.drop(['tFinal','nx','nSteps','tol','alphaErr2','relAlphaErr2',
                                                'relUErr2','ErrorCheckRunTime','VTKRunTime','ParameterSpace',
                                                'TotalTime'],axis=1).reset_index(drop=True)
ODECO_multiple_data['numSensors'] = ODECO_multiple_data['numSensors'].replace(3,'Multiple with 3 sensors')
ODECO_multiple_data['numSensors'] = ODECO_multiple_data['numSensors'].replace(11,'Multiple with 11 sensors')
ODECO_multiple_data = ODECO_multiple_data.rename(columns={'numSensors':'Source',
                                                          'uErr2':'uErr',
                                                          'KKTRunTime':'Runtime'})[col_order]
display(ODECO_multiple_data)


,Source,Re,uErr,Runtime
0,Multiple with 3 sensors,60,0.000950,2.09658
1,Multiple with 11 sensors,60,0.013650,3.81729
2,Multiple with 3 sensors,75,0.000408,2.11086
3,Multiple with 11 sensors,75,0.000938,3.85103
4,Multiple with 3 sensors,80,0.000417,2.08081
5,Multiple with 11 sensors,80,0.000707,3.78796
6,Multiple with 3 sensors,100,0.000529,2.06190
7,Multiple with 11 sensors,100,0.000354,3.78731


In [120]:
# put it all together
total_data = viento_data.append(inv_data).append(ODECO_multiple_data).reset_index(drop=True).sort_values('Re')
display(total_data)
# Calculate the increase in speed
test = total_data.groupby('Re').apply(lambda x: x.iloc[0]['Runtime']/x['Runtime']).reset_index(level='Re',drop=True)
# display(test)
# display(test.reset_index(level='level_1',drop=True))
total_data['Speed Up'] = test
total_data = total_data.reset_index(drop=True)
total_data.to_csv(ODECO_multiple_dir+'SpeedUp_numSens{3,11}_nx72.csv',index=False)
display(total_data)

,Source,Re,uErr,Runtime
0,Viento,60,0.000015,1793.04000
4,Inverse Problem,60,0.000258,128.36500
8,Multiple with 3 sensors,60,0.000950,2.09658
9,Multiple with 11 sensors,60,0.013650,3.81729
1,Viento,75,0.000016,1798.03000
5,Inverse problem,75,NaN,NaN
10,Multiple with 3 sensors,75,0.000408,2.11086
11,Multiple with 11 sensors,75,0.000938,3.85103
2,Viento,80,0.000017,1778.68000
6,Inverse Problem,80,0.000263,128.86000


,Source,Re,uErr,Runtime,Speed Up
0,Viento,60,0.000015,1793.04000,1.000000
1,Inverse Problem,60,0.000258,128.36500,13.968294
2,Multiple with 3 sensors,60,0.000950,2.09658,855.221361
3,Multiple with 11 sensors,60,0.013650,3.81729,469.715426
4,Viento,75,0.000016,1798.03000,1.000000
5,Inverse problem,75,NaN,NaN,NaN
6,Multiple with 3 sensors,75,0.000408,2.11086,851.799740
7,Multiple with 11 sensors,75,0.000938,3.85103,466.895869
8,Viento,80,0.000017,1778.68000,1.000000
9,Inverse Problem,80,0.000263,128.86000,13.803197


In [43]:
s1 = pd.Series([5, 6, 7])
s2 = pd.Series([7, 8, 9])

df = pd.DataFrame([list(s1), list(s2)],  columns =  ["A", "B", "C"])
df

,A,B,C
0,5,6,7
1,7,8,9


In [45]:
display(df.index)
display(df.index+1)

RangeIndex(start=0, stop=2, step=1)

RangeIndex(start=1, stop=3, step=1)

In [36]:
df.loc[-1] = [2, 3, 4]  # adding a row
df.index = df.index + 1  # shifting index
df = df.sort_index()  # sorting by index
df

,A,B,C
0,2,3,4
1,5,6,7
2,7,8,9
